In [31]:
#this combines the files together in the directories
import pandas as pd
import numpy as np
import pathlib2 as pl2

def combine_files_together():
    base_path = "data/"
    elms_path = "ELMS"
    lmc_cup_path = "LeMansCup"
    fiawec_path = "FIAWEC"
    imsa_path = "IMSA"

    #file_paths = [fiawec_path, elms_path, lmc_cup_path, imsa_path]
    file_paths = [fiawec_path, imsa_path]
    #file_paths = [lmc_cup_path]
    #combine the dfs together
    for f in file_paths:
        curr_dir = base_path + f
        full_df = pd.DataFrame()
        current_circuit = ""
        ps = pl2.Path(curr_dir)
        
        rows = (pd.read_csv(p, index_col=0, dtype=str) for p in ps.glob('*.csv'))
        full_df = pd.concat(rows)
        full_csv_loc = f + "_full_data.csv"
        full_df = full_df.dropna(axis=1, how = "all")
        full_df = full_df.reset_index(drop=True)
        full_df.columns = full_df.columns.str.strip()
        full_df.columns = full_df.columns.str.lower()
        full_df.to_csv(full_csv_loc)
combine_files_together()

In [32]:
#load all the combined dfs
def load_all_dfs():
    fia_wec_df = pd.read_csv('FIAWEC_full_data.csv', index_col = 0, dtype=str)
    imsa_df = pd.read_csv('IMSA_full_data.csv', index_col = 0,dtype=str)
    #elms_df = pd.read_csv('ELMS_full_data.csv', index_col = 0)
    #lmc_df = pd.read_csv('LeMansCup_full_data.csv', index_col = 0)
    #full_df = pd.concat([fia_wec_df, imsa_df, elms_df, lmc_df])
    full_df = pd.concat([fia_wec_df, imsa_df])
    full_df = full_df.reset_index(drop=True)
    return full_df
full_df = load_all_dfs()
full_df.to_csv('full_data_2023.csv')

In [33]:
full_df['key'] = full_df['season'].map(str) + "_" + full_df['championship'] + "_" + full_df['round'].map(str) \
        + "_" + full_df['session_type'] + "_" + \
                full_df['number'].map(str) +  "_" + \
                full_df['lap_number'].map(str)
full_df[full_df['number'] == '01']

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,s1_seconds,s2_seconds,s3_seconds,vehicle,championship,session_type,season,circuit,round,key
10119,01,2,1,13:38.866,0,B,2:06.049,0,1:00.754,0,...,NaN,NaN,NaN,Cadillac V-LMDh,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,2023_IMSA_2_PRACTICE 1_01_1
10120,01,2,2,6:17.456,0,B,54.603,0,56.445,0,...,NaN,NaN,NaN,Cadillac V-LMDh,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,2023_IMSA_2_PRACTICE 1_01_2
10121,01,2,3,2:11.805,0,NaN,51.017,0,50.800,0,...,NaN,NaN,NaN,Cadillac V-LMDh,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,2023_IMSA_2_PRACTICE 1_01_3
10122,01,2,4,1:39.473,0,NaN,23.671,0,46.826,0,...,NaN,NaN,NaN,Cadillac V-LMDh,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,2023_IMSA_2_PRACTICE 1_01_4
10123,01,2,5,1:38.641,0,NaN,23.883,0,45.462,0,...,NaN,NaN,NaN,Cadillac V-LMDh,IMSA,PRACTICE 1,2023,Daytona International Speedway,2,2023_IMSA_2_PRACTICE 1_01_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74692,01,1,237,1:51.293,0,NaN,10.026,0,1:10.263,0,...,NaN,NaN,NaN,Cadillac V-Series R,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_01_237
74693,01,1,238,1:51.346,0,NaN,10.424,0,1:09.880,0,...,NaN,NaN,NaN,Cadillac V-Series R,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_01_238
74694,01,1,239,1:50.571,0,NaN,10.337,0,1:09.911,0,...,NaN,NaN,NaN,Cadillac V-Series R,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_01_239
74695,01,1,240,1:49.164,0,NaN,10.015,0,1:08.746,0,...,NaN,NaN,NaN,Cadillac V-Series R,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_01_240


In [34]:
full_df = full_df.drop_duplicates()

In [35]:
df_counts = full_df.groupby('key').count()
df_counts[df_counts['number'] > 1]

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,flag_at_fl,s1_seconds,s2_seconds,s3_seconds,vehicle,championship,session_type,season,circuit,round
key,,,,,,,,,,,,,,,,,,,,,


In [36]:
def fix_columns():
    #these help fix the team, group, class
    full_df['fix_team'] = np.where(full_df['team'].isna(), np.where(full_df['team_x'].isna(), full_df['team_y'], full_df['team_x']),full_df['team'])
    full_df[full_df['fix_team'].isna()]
    full_df['fix_class'] = np.where(full_df['class'].isna(), np.where(full_df['class_x'].isna(), full_df['class_y'], full_df['class_x']),full_df['class'])
    full_df[full_df['fix_class'].isna()]
    full_df['fix_group'] = np.where(full_df['group'].isna(), np.where(full_df['group_x'].isna(), full_df['group_y'], full_df['group_x']),full_df['group'])
    full_df[full_df['fix_group'].isna()]

    full_df['team'] = full_df['fix_team']
    full_df['class'] = full_df['fix_class']
    full_df['group'] = full_df['fix_group']

    full_df_fixed_columns = full_df.drop(['team_x', 'team_y', 'fix_team', 'group_x', 'group_y', 'fix_group', 'class_x', 'class_y','fix_class'], axis=1)
full_df_fixed_columns = full_df

In [37]:
#okay so the next thing to prob fix to drop porsche,ford,t6,t7, not used really for anything
#also fix s1,s2,s3, pit_time and convert to seconds
full_df_fixed_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89652 entries, 0 to 89651
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   number                       89652 non-null  object
 1   driver_number                89652 non-null  object
 2   lap_number                   89652 non-null  object
 3   lap_time                     89652 non-null  object
 4   lap_improvement              89652 non-null  object
 5   crossing_finish_line_in_pit  7247 non-null   object
 6   s1                           89628 non-null  object
 7   s1_improvement               89652 non-null  object
 8   s2                           89518 non-null  object
 9   s2_improvement               89652 non-null  object
 10  s3                           89533 non-null  object
 11  s3_improvement               89652 non-null  object
 12  kph                          89652 non-null  object
 13  elapsed                      89

In [38]:
def fix_full_df_without_lm():
    #drop the ones we don't need like the le mans specific ones
    full_df_without_lm = full_df_fixed_columns.drop(['porsche', 'ford', 't6', 't7'], axis=1)

    #here we're going to fix the sectors, this is just showing why we would need to fix those
    wanted_columns = ['s1', 's2', 's3', 'pit_time', 'elapsed',  's1_seconds', 's2_seconds', 's3_seconds', 's1_large', 's2_large', 's3_large']
    full_df_sectors = full_df_without_lm[wanted_columns]
    full_df_sectors.head(10)    
full_df_without_lm = full_df

In [39]:
full_df_without_large = full_df_without_lm.drop(['s1_large', 's2_large', 's3_large'], axis=1)
full_df_without_large.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89652 entries, 0 to 89651
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   number                       89652 non-null  object
 1   driver_number                89652 non-null  object
 2   lap_number                   89652 non-null  object
 3   lap_time                     89652 non-null  object
 4   lap_improvement              89652 non-null  object
 5   crossing_finish_line_in_pit  7247 non-null   object
 6   s1                           89628 non-null  object
 7   s1_improvement               89652 non-null  object
 8   s2                           89518 non-null  object
 9   s2_improvement               89652 non-null  object
 10  s3                           89533 non-null  object
 11  s3_improvement               89652 non-null  object
 12  kph                          89652 non-null  object
 13  elapsed                      89

In [40]:
#this we use to convert anything to ms
def convert_to_s(x):
    if(x == np.nan):
        return x
    hour = 0
    minutes = 0
    try:
        hour, minutes, second = x.split(":")
    except ValueError:
        hour = 0
        try:
            minutes, second = x.split(":")
        except ValueError:
            hour = 0
            minutes = 0
            second = x
    except AttributeError:
        print(x)
    hour_seconds = int(hour)*60*60
    if hour == 0 and minutes == 0:
        return x
    else:
        ms = 0
        try:
            seconds, ms = second.split(".")
        except ValueError:
            seconds = second
    minute_seconds = (int(minutes)*60 + int(seconds) + int(hour_seconds))*1000 
    total_ms = minute_seconds + int(ms)
    return total_ms/1000

#this uses the method to convert all of the large ones (h:m:s.ms to seconds)
def convert_df_cols_to_seconds(full_df):
    full_df['lap_time_seconds'] = full_df.lap_time.map(convert_to_s)
    full_df['s1_seconds'] = full_df.s1.map(convert_to_s)
    full_df['s2_seconds'] = full_df.s2.map(convert_to_s)
    full_df['s3_seconds'] = full_df.s3.map(convert_to_s)
    full_df['elapsed_seconds'] = full_df.elapsed.map(convert_to_s)
    full_df['pit_time_seconds'] = full_df.pit_time.map(convert_to_s)
    return full_df
full_df_fix_seconds = convert_df_cols_to_seconds(full_df_without_large)
full_df_fix_seconds 

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,vehicle,championship,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds
0,10,2,1,2:16.069,0,B,42.370,0,48.275,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_1,136.069,189.560,24.327
1,10,2,2,2:02.784,0,NaN,36.446,0,42.425,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_2,122.784,312.344,NaN
2,10,2,3,1:59.775,0,NaN,36.910,0,41.287,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_3,119.775,432.119,NaN
3,10,2,4,1:54.197,0,NaN,33.920,0,39.316,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_4,114.197,546.316,NaN
4,10,2,5,1:52.755,0,NaN,33.456,0,39.414,0,...,Oreca 07 - Gibson,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_5,112.755,659.071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89647,96,2,297,3:59.239,0,NaN,17.397,0,2:30.515,0,...,BMW M4 GT3,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_297,239.239,42723.284,NaN
89648,96,2,298,3:31.004,0,NaN,26.156,0,2:12.811,0,...,BMW M4 GT3,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_298,211.004,42934.288,NaN
89649,96,2,299,2:06.232,0,NaN,11.924,0,1:19.116,0,...,BMW M4 GT3,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_299,126.232,43060.520,NaN
89650,96,2,300,2:04.806,0,NaN,11.953,0,1:18.092,0,...,BMW M4 GT3,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_300,124.806,43185.326,NaN


In [41]:
full_df_fix_seconds['session'] = full_df_fix_seconds['session_type']
full_df_fix_seconds

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,championship,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds,session
0,10,2,1,2:16.069,0,B,42.370,0,48.275,0,...,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_1,136.069,189.560,24.327,FREE PRACTICE 1
1,10,2,2,2:02.784,0,NaN,36.446,0,42.425,0,...,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_2,122.784,312.344,NaN,FREE PRACTICE 1
2,10,2,3,1:59.775,0,NaN,36.910,0,41.287,0,...,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_3,119.775,432.119,NaN,FREE PRACTICE 1
3,10,2,4,1:54.197,0,NaN,33.920,0,39.316,0,...,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_4,114.197,546.316,NaN,FREE PRACTICE 1
4,10,2,5,1:52.755,0,NaN,33.456,0,39.414,0,...,FIAWEC,FREE PRACTICE 1,2023,SEBRING,1,2023_FIAWEC_1_FREE PRACTICE 1_10_5,112.755,659.071,NaN,FREE PRACTICE 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89647,96,2,297,3:59.239,0,NaN,17.397,0,2:30.515,0,...,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_297,239.239,42723.284,NaN,RACE
89648,96,2,298,3:31.004,0,NaN,26.156,0,2:12.811,0,...,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_298,211.004,42934.288,NaN,RACE
89649,96,2,299,2:06.232,0,NaN,11.924,0,1:19.116,0,...,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_299,126.232,43060.520,NaN,RACE
89650,96,2,300,2:04.806,0,NaN,11.953,0,1:18.092,0,...,IMSA,RACE,2023,Sebring International Raceway,4,2023_IMSA_4_RACE_96_300,124.806,43185.326,NaN,RACE


In [42]:
def map_sessions(s):
    if ('QUALIFYING' in s):
        if ('RACE' in s):
            return 'RACE'
        return 'QUALIFYING'
    elif('SESSION' in s or 'TEST' in s):
        return 'TEST'
    elif ('PRACTICE' in s):
        return 'FREE PRACTICE'
    elif ('RACE' in s):
        return 'RACE'
    else:
        return s
#maps the sessions to session_types

full_df_fix_seconds['session_type'] = full_df_fix_seconds['session'].map(map_sessions)
full_df_test = full_df_fix_seconds.groupby(['session','session_type']).mean()
full_df_test

,,lap_time_seconds,elapsed_seconds,pit_time_seconds
session,session_type,,,
FREE PRACTICE 1,FREE PRACTICE,135.938467,1959.088954,165.773348
FREE PRACTICE 2,FREE PRACTICE,168.386366,2124.253199,290.982792
FREE PRACTICE 3,FREE PRACTICE,146.454174,1801.659435,125.958940
PRACTICE 1,FREE PRACTICE,205.787678,2655.126420,354.767167
PRACTICE 2,FREE PRACTICE,175.315200,2824.742088,309.558336
PRACTICE 3,FREE PRACTICE,184.081983,3100.184866,306.329102
PRACTICE 4,FREE PRACTICE,151.337878,1559.560469,161.675014
PRACTICE 5,FREE PRACTICE,140.967225,628.830875,107.664154
QUALIFYING,QUALIFYING,317.800021,1715.617532,1186.195112


In [43]:
full_df_fix_seconds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89652 entries, 0 to 89651
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   number                       89652 non-null  object 
 1   driver_number                89652 non-null  object 
 2   lap_number                   89652 non-null  object 
 3   lap_time                     89652 non-null  object 
 4   lap_improvement              89652 non-null  object 
 5   crossing_finish_line_in_pit  7247 non-null   object 
 6   s1                           89628 non-null  object 
 7   s1_improvement               89652 non-null  object 
 8   s2                           89518 non-null  object 
 9   s2_improvement               89652 non-null  object 
 10  s3                           89533 non-null  object 
 11  s3_improvement               89652 non-null  object 
 12  kph                          89652 non-null  object 
 13  elapsed         

In [44]:
full_df_fix_seconds[(full_df_fix_seconds['team'].isna())]

,number,driver_number,lap_number,lap_time,lap_improvement,crossing_finish_line_in_pit,s1,s1_improvement,s2,s2_improvement,...,championship,session_type,season,circuit,round,key,lap_time_seconds,elapsed_seconds,pit_time_seconds,session


In [45]:
full_df_fix_seconds_columns = full_df_fix_seconds.drop(['s1', 's2', 's3', 'lap_time', 'elapsed', 'pit_time'], axis=1)

In [46]:
full_df_fix_seconds_columns.to_csv('alkamel_data_full.csv', index=False)

In [47]:
full_df = pd.read_csv('alkamel_data_full.csv')
wanted_columns = ['event', 'circuit']
full_df['event'] = full_df['circuit']
full_df_group = full_df.groupby(['circuit','event']).mean().reset_index()
full_df_group[wanted_columns].to_csv('new_2022_events.csv',index=False)

C:\Users\trist\AppData\Local\Temp/ipykernel_21784/3655738928.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('alkamel_data_full.csv')


In [54]:
events = pd.read_csv('new_2022_events.csv')
full_df_w_event = pd.merge(left = full_df, right = events, on = 'circuit')
full_df_w_event['event'] = full_df_w_event['event_x']
full_df_w_event['circuit'] = full_df_w_event['event_y']
full_df_w_event = full_df_w_event.drop(['event_x', 'event_y'], axis=1)
full_df_w_event.groupby(['circuit','event']).mean()

number  \
circuit                        event                                       
Daytona International Speedway Daytona International Speedway  41.809285   
                               ROAR Before the 24              42.745964   
Sebring International Raceway  SEBRING                         70.072932   
                               Sebring February Test           40.721716   
                               Sebring International Raceway   40.187415   

                                                               driver_number  \
circuit                        event                                           
Daytona International Speedway Daytona International Speedway       2.410101   
                               ROAR Before the 24                   2.274891   
Sebring International Raceway  SEBRING                              1.964621   
                               Sebring February Test                1.977737   
                               Sebring International Raceway        1.968714   

                                                               lap_number  \
circuit                        event                                        
Daytona International Speedway Daytona International Speedway  302.025436   
                               ROAR Before the 24               16.233769   
Sebring International Raceway  SEBRING                          83.713509   
                               Sebring February Test            21.915199   
                               Sebring International Raceway   116.464486   

                                                               lap_improvement  \
circuit                        event                                             
Daytona International Speedway Daytona International Speedway         0.013074   
                               ROAR Before the 24                     0.080005   
Sebring International Raceway  SEBRING                                0.036466   
                               Sebring February Test                  0.054038   
                               Sebring International Raceway          0.031995   

                                                               s1_improvement  \
circuit                        event                                            
Daytona International Speedway Daytona International Speedway        0.024983   
                               ROAR Before the 24                    0.127725   
Sebring International Raceway  SEBRING                               0.055836   
                               Sebring February Test                 0.090063   
                               Sebring International Raceway         0.047537   

                                                               s2_improvement  \
circuit                        event                                            
Daytona International Speedway Daytona International Speedway        0.026126   
                               ROAR Before the 24                    0.129021   
Sebring International Raceway  SEBRING                               0.057516   
                               Sebring February Test                 0.090265   
                               Sebring International Raceway         0.049157   

                                                               s3_improvement  \
circuit                        event                                            
Daytona International Speedway Daytona International Speedway        0.025414   
                               ROAR Before the 24                    0.130317   
Sebring International Raceway  SEBRING                               0.058701   
                               Sebring February Test                 0.090468   
                               Sebring International Raceway         0.049056   

                                                                      kph  \
circuit                        event                                        

In [55]:
full_df_w_event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89652 entries, 0 to 89651
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   number                       89652 non-null  int64  
 1   driver_number                89652 non-null  int64  
 2   lap_number                   89652 non-null  int64  
 3   lap_improvement              89652 non-null  int64  
 4   crossing_finish_line_in_pit  7247 non-null   object 
 5   s1_improvement               89652 non-null  int64  
 6   s2_improvement               89652 non-null  int64  
 7   s3_improvement               89652 non-null  int64  
 8   kph                          89652 non-null  float64
 9   hour                         89652 non-null  object 
 10  top_speed                    10119 non-null  float64
 11  driver_name                  89652 non-null  object 
 12  class                        89652 non-null  object 
 13  group           

In [56]:
def remap_columns():
    full_df_w_event['driver_number'] = full_df_w_event['driver_number'].map(int)
    full_df_w_event['lap_number'] = full_df_w_event['lap_number'].map(int)
    full_df_w_event['lap_improvement'] = full_df_w_event['lap_improvement'].map(int)
    full_df_w_event['s1_improvement'] = full_df_w_event['s1_improvement'].map(int)
    full_df_w_event['s2_improvement'] = full_df_w_event['s2_improvement'].map(int)
    full_df_w_event['s3_improvement'] = full_df_w_event['s3_improvement'].map(int)
    full_df_w_event['kph'] = full_df_w_event['kph'].map(float)
    full_df_w_event['top_speed'] = full_df_w_event['top_speed'].map(float)
    full_df_w_event['round'] = full_df_w_event['round'].map(int)
    full_df_w_event['lap_time_seconds'] = full_df_w_event['driver_number'].map(float)
    full_df_w_event['s1_seconds'] = full_df_w_event['s1_seconds'].map(float)
    full_df_w_event['s2_seconds'] = full_df_w_event['s2_seconds'].map(float)
    full_df_w_event['s3_seconds'] = full_df_w_event['s3_seconds'].map(float)
    full_df_w_event['elapsed_seconds'] = full_df_w_event['elapsed_seconds'].map(float)
    full_df_w_event['pit_time_seconds'] = full_df_w_event['pit_time_seconds'].map(float)
full_df_w_event.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89652 entries, 0 to 89651
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   number                       89652 non-null  int64  
 1   driver_number                89652 non-null  int64  
 2   lap_number                   89652 non-null  int64  
 3   lap_improvement              89652 non-null  int64  
 4   crossing_finish_line_in_pit  7247 non-null   object 
 5   s1_improvement               89652 non-null  int64  
 6   s2_improvement               89652 non-null  int64  
 7   s3_improvement               89652 non-null  int64  
 8   kph                          89652 non-null  float64
 9   hour                         89652 non-null  object 
 10  top_speed                    10119 non-null  float64
 11  driver_name                  89652 non-null  object 
 12  class                        89652 non-null  object 
 13  group           

In [57]:
full_df_w_event.to_csv('alkamel_data_2023.csv', index=False)

In [52]:
full_df_w_event.columns

Index(['number', 'driver_number', 'lap_number', 'lap_improvement',
       'crossing_finish_line_in_pit', 's1_improvement', 's2_improvement',
       's3_improvement', 'kph', 'hour', 'top_speed', 'driver_name', 'class',
       'group', 'team', 'manufacturer', 'flag_at_fl', 's1_seconds',
       's2_seconds', 's3_seconds', 'vehicle', 'championship', 'session_type',
       'season', 'circuit', 'round', 'key', 'lap_time_seconds',
       'elapsed_seconds', 'pit_time_seconds', 'session', 'event'],
      dtype='object')